In [1]:
import gym
from env.WarthogEnv import WarthogEnv
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
%matplotlib
from matplotlib import pyplot as plt
from stable_baselines3 import PPO

Using matplotlib backend: TkAgg


In [ ]:
#env = DummyVecEnv([lambda: WarthogEnv('unity_remote.txt')])
env = WarthogEnv('unity_remote.txt')
x = []
y = []
env.reset()
for i in range(0,5000):
    action = [0.5,0.1]
    obs, reward, done, info = env.step(action)
    env.render()
    #x.append(info[0][0])
    #y.append(info[0][1])
    if done:
        #print("resetting")
        env.reset()

In [ ]:
env = WarthogEnv('unity_remote.txt')
#obs = env.reset()
#print(obs)
model = PPO('MlpPolicy', env, verbose=1)

In [ ]:
model.learn(total_timesteps=1e7)

In [ ]:
x = []
y = []
env.reset()
for i in range(0,1000):
    action = [0.5,0.1]
    obs, reward, done, info = env.step(action)
    env.render()
    #x.append(info[0][0])kkkj
    #y.append(info[0][1])
    if done:
        print("resetting")
        env.reset()
    #print(info)


In [ ]:
print(env.num_waypoints)

In [ ]:
plt.plot(x,y,'+r')
plt.ylim(-3, 9)
plt.xlim(9, 21)

In [4]:
model.save("bicycle6")

In [2]:
env = WarthogEnv('unity_remote.txt')

/home/akhil/anaconda3/envs/rl/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
model1 = PPO('MlpPolicy', env, verbose=1)
#for learning uncomment
model = PPO('MlpPolicy', env, verbose=1)
# model.load('./first_pytorch_multiplication_reward.zip')
model = PPO.load('bicycle4')
model.env = model1.env
model.learn(total_timesteps=1e7)

Using cuda device
Wrapping the env in a DummyVecEnv.
Using cuda device
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 34   |
|    iterations      | 1    |
|    time_elapsed    | 58   |
|    total_timesteps | 2048 |
-----------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 33        |
|    iterations           | 2         |
|    time_elapsed         | 123       |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.2561556 |
|    clip_fraction        | 0.507     |
|    clip_range           | 0.2       |
|    entropy_loss         | 2.69      |
|    explained_variance   | 0.997     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.287     |
|    n_updates            | 10470     |
|    policy_gradient_loss | 0.000208  |
|    std                  | 0.0687    |
|   

----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 13         |
|    time_elapsed         | 804        |
|    total_timesteps      | 26624      |
| train/                  |            |
|    approx_kl            | 0.13595472 |
|    clip_fraction        | 0.463      |
|    clip_range           | 0.2        |
|    entropy_loss         | 2.66       |
|    explained_variance   | 0.989      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.831      |
|    n_updates            | 10580      |
|    policy_gradient_loss | 0.00663    |
|    std                  | 0.0691     |
|    value_loss           | 2.82       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 14         |
|    time_elapsed         | 866        |
|    total_times

----------------------------------------
| time/                   |            |
|    fps                  | 32         |
|    iterations           | 24         |
|    time_elapsed         | 1489       |
|    total_timesteps      | 49152      |
| train/                  |            |
|    approx_kl            | 0.11580303 |
|    clip_fraction        | 0.423      |
|    clip_range           | 0.2        |
|    entropy_loss         | 2.71       |
|    explained_variance   | 0.995      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.593      |
|    n_updates            | 10690      |
|    policy_gradient_loss | 0.00425    |
|    std                  | 0.0675     |
|    value_loss           | 1.71       |
----------------------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 33        |
|    iterations           | 25        |
|    time_elapsed         | 1550      |
|    total_timesteps 

-----------------------------------------
| time/                   |             |
|    fps                  | 32          |
|    iterations           | 35          |
|    time_elapsed         | 2173        |
|    total_timesteps      | 71680       |
| train/                  |             |
|    approx_kl            | 0.112026036 |
|    clip_fraction        | 0.369       |
|    clip_range           | 0.2         |
|    entropy_loss         | 2.7         |
|    explained_variance   | 0.991       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.12        |
|    n_updates            | 10800       |
|    policy_gradient_loss | -0.00487    |
|    std                  | 0.0683      |
|    value_loss           | 3.16        |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 32         |
|    iterations           | 36         |
|    time_elapsed         | 2234      

-----------------------------------------
| time/                   |             |
|    fps                  | 33          |
|    iterations           | 46          |
|    time_elapsed         | 2844        |
|    total_timesteps      | 94208       |
| train/                  |             |
|    approx_kl            | 0.089891426 |
|    clip_fraction        | 0.421       |
|    clip_range           | 0.2         |
|    entropy_loss         | 2.7         |
|    explained_variance   | 0.996       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.398       |
|    n_updates            | 10910       |
|    policy_gradient_loss | 0.00281     |
|    std                  | 0.0683      |
|    value_loss           | 1.35        |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 33         |
|    iterations           | 47         |
|    time_elapsed         | 2904      

...

----------------------------------------
| time/                   |            |
|    fps                  | 34         |
|    iterations           | 277        |
|    time_elapsed         | 16264      |
|    total_timesteps      | 567296     |
| train/                  |            |
|    approx_kl            | 0.18090719 |
|    clip_fraction        | 0.508      |
|    clip_range           | 0.2        |
|    entropy_loss         | 2.72       |
|    explained_variance   | 0.991      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.803      |
|    n_updates            | 13220      |
|    policy_gradient_loss | 0.00865    |
|    std                  | 0.0653     |
|    value_loss           | 2.25       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 34          |
|    iterations           | 278         |
|    time_elapsed         | 16326       |
|    total_

----------------------------------------
| time/                   |            |
|    fps                  | 34         |
|    iterations           | 288        |
|    time_elapsed         | 16954      |
|    total_timesteps      | 589824     |
| train/                  |            |
|    approx_kl            | 0.28676283 |
|    clip_fraction        | 0.379      |
|    clip_range           | 0.2        |
|    entropy_loss         | 2.66       |
|    explained_variance   | 0.889      |
|    learning_rate        | 0.0003     |
|    loss                 | 30.7       |
|    n_updates            | 13330      |
|    policy_gradient_loss | 0.00213    |
|    std                  | 0.0675     |
|    value_loss           | 35.9       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 34          |
|    iterations           | 289         |
|    time_elapsed         | 17020       |
|    total_

----------------------------------------
| time/                   |            |
|    fps                  | 34         |
|    iterations           | 299        |
|    time_elapsed         | 17655      |
|    total_timesteps      | 612352     |
| train/                  |            |
|    approx_kl            | 0.15586853 |
|    clip_fraction        | 0.461      |
|    clip_range           | 0.2        |
|    entropy_loss         | 2.66       |
|    explained_variance   | 0.987      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.801      |
|    n_updates            | 13440      |
|    policy_gradient_loss | -0.00349   |
|    std                  | 0.067      |
|    value_loss           | 2.89       |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 34         |
|    iterations           | 300        |
|    time_elapsed         | 17722      |
|    total_times

----------------------------------------
| time/                   |            |
|    fps                  | 34         |
|    iterations           | 310        |
|    time_elapsed         | 18376      |
|    total_timesteps      | 634880     |
| train/                  |            |
|    approx_kl            | 0.08195022 |
|    clip_fraction        | 0.386      |
|    clip_range           | 0.2        |
|    entropy_loss         | 2.69       |
|    explained_variance   | 0.987      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.922      |
|    n_updates            | 13550      |
|    policy_gradient_loss | -0.0049    |
|    std                  | 0.0656     |
|    value_loss           | 4.71       |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 34          |
|    iterations           | 311         |
|    time_elapsed         | 18438       |
|    total_

KeyboardInterrupt: 